In [ ]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [ ]:
cd MyDrive/MyDrive/deepLearning/dataset

/content/MyDrive/MyDrive/deepLearning/dataset


In [ ]:
import pandas as pd
anx = pd.read_csv('anx_c.csv')
bi = pd.read_csv('bi_c.csv')
sch = pd.read_csv('sch_c.csv')
adhd = pd.read_csv('adhd_c.csv')
dep = pd.read_csv('dep_c.csv')
nonM = pd.read_csv('nonM_c.csv')

In [ ]:
minimum = min(len(bi),len(nonM),len(anx),len(adhd),len(dep),len(sch))

In [ ]:
dep_s = dep.sample(n = minimum,random_state = 1)
nonM_s = nonM.sample(n = minimum,random_state = 1)
anx_s = anx.sample(n = minimum,random_state = 1)
sch_s = sch.sample(n = minimum,random_state = 1)
bi_s = bi.sample(n = minimum,random_state = 1)
adhd_s = adhd.sample(n = minimum,random_state = 1)

In [ ]:
pip install http://h2o-release.s3.amazonaws.com/h2o/master/5387/Python/h2o-3.33.1.5387-py2.py3-none-any.whl

     |████████████████████████████████| 164.8 MB 1.4 MB/s 


In [ ]:
data = pd.concat([dep_s[['content','class']],
                  nonM_s[['content','class']],
                  anx_s[['content','class']],
                  bi_s[['content','class']],
                  sch_s[['content','class']],
                  adhd_s[['content','class']]])

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(data,test_size=0.3,random_state=1,stratify = data['class'])


print(train.groupby('class').count())
print(test.groupby('class').count())

       content
class         
adhd      9207
anx       9209
bi        9209
dep       9208
nonM      9209
sch       9208
       content
class         
adhd      3947
anx       3946
bi        3946
dep       3947
nonM      3946
sch       3947


In [ ]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpv5bdb0bb
  JVM stdout: /tmp/tmpv5bdb0bb/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpv5bdb0bb/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.33.1.5387
H2O_cluster_version_age:,4 months and 26 days !!!
H2O_cluster_name:,H2O_from_python_unknownUser_q1r5tt
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
grouped = test.group_by('class')
h2o.as_list(grouped.get_frame(),use_pandas=False)

[['class'], ['adhd'], ['anx'], ['bi'], ['dep'], ['nonM'], ['sch']]

In [ ]:
train['class'] = train['class'].asfactor()
test['class'] = test['class']

In [ ]:
from h2o.estimators.word2vec import H2OWord2vecEstimator
import os
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
    data_path = docker_data_path
else:
    data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

In [ ]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [ ]:
words = tokenize(train["content"])

In [ ]:
test_words = tokenize(test["content"])

In [ ]:
w2v_model = H2OWord2vecEstimator(vec_size = 128, model_id = "w2v_mh.hex")
w2v_model.train(training_frame=words)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [ ]:
content_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")
test_content_vecs=w2v_model.transform(test_words,aggregate_method = "AVERAGE")

In [ ]:
emb_train = train.cbind(content_vecs)
emb_test = test.cbind(test_content_vecs)

In [ ]:
predictors = content_vecs.names
response = 'class'

In [ ]:
type(predictors)

list

In [ ]:
from h2o.automl import H2OAutoML
import time
start = time.time()
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=30,seed = 1,exclude_algos=['XGBoost'],max_runtime_secs = 1200,nfolds=3,keep_cross_validation_predictions=True,keep_cross_validation_models=True,keep_cross_validation_fold_assignment=True)
aml.train(x=predictors, y=response, training_frame=emb_train,leaderboard_frame=emb_test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse,auc,aucpr
StackedEnsemble_AllModels_AutoML_20210820_123811,0.229445,0.698071,0.469207,0.220155,nan,nan
StackedEnsemble_BestOfFamily_AutoML_20210820_123811,0.235104,0.708248,0.472307,0.223074,nan,nan
DeepLearning_1_AutoML_20210820_123811,0.236963,0.72049,0.463793,0.215104,nan,nan
DeepLearning_grid__2_AutoML_20210820_123811_model_1,0.254149,0.81388,0.472506,0.223262,nan,nan
DeepLearning_grid__1_AutoML_20210820_123811_model_1,0.258289,0.963915,0.475264,0.225876,nan,nan
GBM_grid__1_AutoML_20210820_123811_model_1,0.295157,0.894475,0.559514,0.313056,nan,nan
DeepLearning_grid__3_AutoML_20210820_123811_model_1,0.310279,1.02252,0.607241,0.368742,nan,nan
GBM_3_AutoML_20210820_123811,0.335404,1.28284,0.706567,0.499236,nan,nan
GBM_2_AutoML_20210820_123811,0.344653,1.30586,0.713758,0.509451,nan,nan
GBM_5_AutoML_20210820_123811,0.345328,1.42091,0.749781,0.562171,nan,nan


In [ ]:
import numpy as np

Pred = h2o.as_list(aml.predict(emb_test),use_pandas=False)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [ ]:
result = [i[0] for i in Pred]
result = result[1:]

In [ ]:
test_label = np.array(h2o.as_list(emb_test['class'],use_pandas=False)[1:]).reshape(1,-1)[0]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(test_label, result))

              precision    recall  f1-score   support

        adhd       0.83      0.85      0.84      3947
         anx       0.76      0.74      0.75      3946
          bi       0.76      0.70      0.73      3946
         dep       0.67      0.74      0.70      3947
        nonM       0.83      0.86      0.84      3946
         sch       0.78      0.74      0.76      3947

    accuracy                           0.77     23679
   macro avg       0.77      0.77      0.77     23679
weighted avg       0.77      0.77      0.77     23679

